# Crear Target

Instalamos / actualizamos librerías

In [1]:
##%%bash 
# ! pip install duckdb
# ! pip install jupysql
# ! pip install duckdb-engine
# ! pip install pandas

In [4]:
# ! pip install --upgrade duckdb
# ! pip install --upgrade jupysql
# ! pip install --upgrade duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/dmeyf.db

Cargamos la tabla generada para la segunda competencia:

In [2]:
%%sql
select
    *   
from competencia_02
limit 2

numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
0           29183981    201901               1            0         1   
1           29184630    201901               1            0         1   

   cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
0            48                 171        8304.31              42993.54   
1            57                 296       18318.76              42597.26   

   mcomisiones  ...  Visa_fultimo_cierre  Visa_mpagado  Visa_mpagospesos  \
0      2501.04  ...                    8           0.0         -35773.32   
1     14986.76  ...                    1           0.0         -67414.82   

   Visa_mpagosdolares  Visa_fechaalta  Visa_mconsumototal  Visa_cconsumos  \
0                2.33            5148            36602.13              54   
1             1149.57            3299            28503.06              10   

   Visa_cadelantosefectivo  Visa_mpagominimo  clase_ternaria  
0                        0           5008.71        CONTINUA  
1                        0           1395.87        CONTINUA  

[2 rows x 155 columns]

In [3]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_02
where numero_de_cliente = 186418975

numero_de_cliente  foto_mes  ctrx_quarter  lag_1_ctrx_quarter
0          186418975    202106            17                 NaN
1          186418975    202107            33                17.0

Lista de variables con las que generaré nuevas variables:

In [4]:
campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 'cseguro_accidentes_personales', 'ccaja_seguridad', 'cpayroll_trx', 'mpayroll', 'mpayroll2', 'cpayroll2_trx', 'ccuenta_debitos_automaticos', 'mcuenta_debitos_automaticos', 'ctarjeta_visa_debitos_automaticos', 'mttarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos', 'mttarjeta_master_debitos_automaticos', 'cpagodeservicios', 'mpagodeservicios', 'cpagomiscuentas', 'mpagomiscuentas', 'ccajeros_propios_descuentos', 'mcajeros_propios_descuentos', 'ctarjeta_visa_descuentos', 'mtarjeta_visa_descuentos', 'ctarjeta_master_descuentos', 'mtarjeta_master_descuentos', 'ccomisiones_mantenimiento', 'mcomisiones_mantenimiento', 'ccomisiones_otras', 'mcomisiones_otras', 'cforex', 'cforex_buy', 'mforex_buy', 'cforex_sell', 'mforex_sell', 'ctransferencias_recibidas', 'mtransferencias_recibidas', 'ctransferencias_emitidas', 'mtransferencias_emitidas', 'cextraccion_autoservicio', 'mextraccion_autoservicio', 'ccheques_depositados', 'mcheques_depositados', 'ccheques_emitidos', 'mcheques_emitidos', 'ccheques_depositados_rechazados', 'mcheques_depositados_rechazados', 'ccheques_emitidos_rechazados', 'mcheques_emitidos_rechazados', 'tcallcenter', 'ccallcenter_transacciones', 'thomebanking', 'chomebanking_transacciones', 'ccajas_transacciones', 'ccajas_consultas', 'ccajas_depositos', 'ccajas_extracciones', 'ccajas_otras', 'catm_trx', 'matm', 'catm_trx_other', 'matm_other', 'ctrx_quarter', 'Master_delinquency', 'Master_status', 'Master_mfinanciacion_limite', 'Master_Fvencimiento', 'Master_Finiciomora', 'Master_msaldototal', 'Master_msaldopesos', 'Master_msaldodolares', 'Master_mconsumospesos', 'Master_mconsumosdolares', 'Master_mlimitecompra', 'Master_madelantopesos', 'Master_madelantodolares', 'Master_fultimo_cierre', 'Master_mpagado', 'Master_mpagospesos', 'Master_mpagosdolares', 'Master_fechaalta', 'Master_mconsumototal', 'Master_cconsumos', 'Master_cadelantosefectivo', 'Master_mpagominimo', 'Visa_delinquency', 'Visa_status', 'Visa_mfinanciacion_limite', 'Visa_Fvencimiento', 'Visa_Finiciomora', 'Visa_msaldototal', 'Visa_msaldopesos', 'Visa_msaldodolares', 'Visa_mconsumospesos', 'Visa_mconsumosdolares', 'Visa_mlimitecompra', 'Visa_madelantopesos', 'Visa_madelantodolares', 'Visa_fultimo_cierre', 'Visa_mpagado', 'Visa_mpagospesos', 'Visa_mpagosdolares', 'Visa_fechaalta', 'Visa_mconsumototal', 'Visa_cconsumos', 'Visa_cadelantosefectivo', 'Visa_mpagominimo']
print(len(campos))

150


In [5]:
# campos
variables = ""
for campo in campos:
  variables += f"{campo},"
print(variables)

active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,mprestamos_hipotecarios,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cinversion1,minversion1_pesos,minversion1_dolares,cinversion2,minversion2,cseguro_vida,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,cpayroll_trx,mpayroll,mpayroll2,cpayroll2_trx,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_

### LAG: Valor del mes anterior de cada columna
### Delta: Calculo la diferencia entre el valor del mes anterior y el presente para cada campo

In [6]:
# Genero la query para cada campo
q_lag = ""
for campo in campos:
  q_lag += f"\n ,lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}  \n ,lag({campo}, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_{campo} \n ,lag({campo}, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_{campo}  \n ,{campo} - lag_1_{campo} as delta_1_{campo}"
#print(q_lag)

In [7]:
with open("C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/q_lag_delta.txt", "w") as archivo:
    archivo.write(q_lag)

In [8]:
%%sql
create or replace table competencia_02_lag_delta as
select
  numero_de_cliente
  ,foto_mes
  ,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,mprestamos_hipotecarios,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cinversion1,minversion1_pesos,minversion1_dolares,cinversion2,minversion2,cseguro_vida,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,cpayroll_trx,mpayroll,mpayroll2,cpayroll2_trx,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,mttarjeta_visa_debitos_automaticos,ctarjeta_master_debitos_automaticos,mttarjeta_master_debitos_automaticos,cpagodeservicios,mpagodeservicios,cpagomiscuentas,mpagomiscuentas,ccajeros_propios_descuentos,mcajeros_propios_descuentos,ctarjeta_visa_descuentos,mtarjeta_visa_descuentos,ctarjeta_master_descuentos,mtarjeta_master_descuentos,ccomisiones_mantenimiento,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,cforex,cforex_buy,mforex_buy,cforex_sell,mforex_sell,ctransferencias_recibidas,mtransferencias_recibidas,ctransferencias_emitidas,mtransferencias_emitidas,cextraccion_autoservicio,mextraccion_autoservicio,ccheques_depositados,mcheques_depositados,ccheques_emitidos,mcheques_emitidos,ccheques_depositados_rechazados,mcheques_depositados_rechazados,ccheques_emitidos_rechazados,mcheques_emitidos_rechazados,tcallcenter,ccallcenter_transacciones,thomebanking,chomebanking_transacciones,ccajas_transacciones,ccajas_consultas,ccajas_depositos,ccajas_extracciones,ccajas_otras,catm_trx,matm,catm_trx_other,matm_other,ctrx_quarter,Master_delinquency,Master_status,Master_mfinanciacion_limite,Master_Fvencimiento,Master_Finiciomora,Master_msaldototal,Master_msaldopesos,Master_msaldodolares,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
  ,lag(active_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_active_quarter  
  ,lag(active_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_active_quarter 
  ,lag(active_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_active_quarter  
  ,active_quarter - lag_1_active_quarter as delta_1_active_quarter
  ,lag(cliente_vip, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_vip  
  ,lag(cliente_vip, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cliente_vip 
  ,lag(cliente_vip, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cliente_vip  
  ,cliente_vip - lag_1_cliente_vip as delta_1_cliente_vip
  ,lag(internet, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_internet  
  ,lag(internet, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_internet 
  ,lag(internet, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_internet  
  ,internet - lag_1_internet as delta_1_internet
  ,lag(cliente_edad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_edad  
  ,lag(cliente_edad, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cliente_edad 
  ,lag(cliente_edad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cliente_edad  
  ,cliente_edad - lag_1_cliente_edad as delta_1_cliente_edad
  ,lag(cliente_antiguedad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_antiguedad  
  ,lag(cliente_antiguedad, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cliente_antiguedad 
  ,lag(cliente_antiguedad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cliente_antiguedad  
  ,cliente_antiguedad - lag_1_cliente_antiguedad as delta_1_cliente_antiguedad
  ,lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad  
  ,lag(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mrentabilidad 
  ,lag(mrentabilidad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mrentabilidad  
  ,mrentabilidad - lag_1_mrentabilidad as delta_1_mrentabilidad
  ,lag(mrentabilidad_annual, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad_annual  
  ,lag(mrentabilidad_annual, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mrentabilidad_annual 
  ,lag(mrentabilidad_annual, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mrentabilidad_annual  
  ,mrentabilidad_annual - lag_1_mrentabilidad_annual as delta_1_mrentabilidad_annual
  ,lag(mcomisiones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones  
  ,lag(mcomisiones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones 
  ,lag(mcomisiones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones  
  ,mcomisiones - lag_1_mcomisiones as delta_1_mcomisiones
  ,lag(mactivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mactivos_margen  
  ,lag(mactivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mactivos_margen 
  ,lag(mactivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mactivos_margen  
  ,mactivos_margen - lag_1_mactivos_margen as delta_1_mactivos_margen
  ,lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen  
  ,lag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpasivos_margen 
  ,lag(mpasivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpasivos_margen  
  ,mpasivos_margen - lag_1_mpasivos_margen as delta_1_mpasivos_margen
  ,lag(cproductos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cproductos  
  ,lag(cproductos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cproductos 
  ,lag(cproductos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cproductos  
  ,cproductos - lag_1_cproductos as delta_1_cproductos
  ,lag(tcuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tcuentas  
  ,lag(tcuentas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_tcuentas 
  ,lag(tcuentas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_tcuentas  
  ,tcuentas - lag_1_tcuentas as delta_1_tcuentas
  ,lag(ccuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccuenta_corriente  
  ,lag(ccuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccuenta_corriente 
  ,lag(ccuenta_corriente, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccuenta_corriente  
  ,ccuenta_corriente - lag_1_ccuenta_corriente as delta_1_ccuenta_corriente
  ,lag(mcuenta_corriente_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente_adicional  
  ,lag(mcuenta_corriente_adicional, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_corriente_adicional 
  ,lag(mcuenta_corriente_adicional, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_corriente_adicional  
  ,mcuenta_corriente_adicional - lag_1_mcuenta_corriente_adicional as delta_1_mcuenta_corriente_adicional
  ,lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente  
  ,lag(mcuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_corriente 
  ,lag(mcuenta_corriente, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_corriente  
  ,mcuenta_corriente - lag_1_mcuenta_corriente as delta_1_mcuenta_corriente
  ,lag(ccaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccaja_ahorro  
  ,lag(ccaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccaja_ahorro 
  ,lag(ccaja_ahorro, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccaja_ahorro  
  ,ccaja_ahorro - lag_1_ccaja_ahorro as delta_1_ccaja_ahorro
  ,lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro  
  ,lag(mcaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro 
  ,lag(mcaja_ahorro, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro  
  ,mcaja_ahorro - lag_1_mcaja_ahorro as delta_1_mcaja_ahorro
  ,lag(mcaja_ahorro_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_adicional  
  ,lag(mcaja_ahorro_adicional, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro_adicional 
  ,lag(mcaja_ahorro_adicional, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro_adicional  
  ,mcaja_ahorro_adicional - lag_1_mcaja_ahorro_adicional as delta_1_mcaja_ahorro_adicional
  ,lag(mcaja_ahorro_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_dolares  
  ,lag(mcaja_ahorro_dolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro_dolares 
  ,lag(mcaja_ahorro_dolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro_dolares  
  ,mcaja_ahorro_dolares - lag_1_mcaja_ahorro_dolares as delta_1_mcaja_ahorro_dolares
  ,lag(cdescubierto_preacordado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cdescubierto_preacordado  
  ,lag(cdescubierto_preacordado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cdescubierto_preacordado 
  ,lag(cdescubierto_preacordado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cdescubierto_preacordado  
  ,cdescubierto_preacordado - lag_1_cdescubierto_preacordado as delta_1_cdescubierto_preacordado
  ,lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo  
  ,lag(mcuentas_saldo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuentas_saldo 
  ,lag(mcuentas_saldo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuentas_saldo  
  ,mcuentas_saldo - lag_1_mcuentas_saldo as delta_1_mcuentas_saldo
  ,lag(ctarjeta_debito, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito  
  ,lag(ctarjeta_debito, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_debito 
  ,lag(ctarjeta_debito, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_debito  
  ,ctarjeta_debito - lag_1_ctarjeta_debito as delta_1_ctarjeta_debito
  ,lag(ctarjeta_debito_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito_transacciones  
  ,lag(ctarjeta_debito_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_debito_transacciones 
  ,lag(ctarjeta_debito_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_debito_transacciones  
  ,ctarjeta_debito_transacciones - lag_1_ctarjeta_debito_transacciones as delta_1_ctarjeta_debito_transacciones
  ,lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mautoservicio  
  ,lag(mautoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mautoservicio 
  ,lag(mautoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mautoservicio  
  ,mautoservicio - lag_1_mautoservicio as delta_1_mautoservicio
  ,lag(ctarjeta_visa, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa  
  ,lag(ctarjeta_visa, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa 
  ,lag(ctarjeta_visa, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa  
  ,ctarjeta_visa - lag_1_ctarjeta_visa as delta_1_ctarjeta_visa
  ,lag(ctarjeta_visa_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_transacciones  
  ,lag(ctarjeta_visa_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa_transacciones 
  ,lag(ctarjeta_visa_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_transacciones  
  ,ctarjeta_visa_transacciones - lag_1_ctarjeta_visa_transacciones as delta_1_ctarjeta_visa_transacciones
  ,lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo  
  ,lag(mtarjeta_visa_consumo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_visa_consumo 
  ,lag(mtarjeta_visa_consumo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_visa_consumo  
  ,mtarjeta_visa_consumo - lag_1_mtarjeta_visa_consumo as delta_1_mtarjeta_visa_consumo
  ,lag(ctarjeta_master, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master  
  ,lag(ctarjeta_master, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_master 
  ,lag(ctarjeta_master, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master  
  ,ctarjeta_master - lag_1_ctarjeta_master as delta_1_ctarjeta_master
  ,lag(ctarjeta_master_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_transacciones  
  ,lag(ctarjeta_master_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_master_transacciones 
  ,lag(ctarjeta_master_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master_transacciones  
  ,ctarjeta_master_transacciones - lag_1_ctarjeta_master_transacciones as delta_1_ctarjeta_master_transacciones
  ,lag(mtarjeta_master_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_consumo  
  ,lag(mtarjeta_master_consumo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_master_consumo 
  ,lag(mtarjeta_master_consumo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_master_consumo  
  ,mtarjeta_master_consumo - lag_1_mtarjeta_master_consumo as delta_1_mtarjeta_master_consumo
  ,lag(cprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_personales  
  ,lag(cprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_personales 
  ,lag(cprestamos_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_personales  
  ,cprestamos_personales - lag_1_cprestamos_personales as delta_1_cprestamos_personales
  ,lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales  
  ,lag(mprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_personales 
  ,lag(mprestamos_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_personales  
  ,mprestamos_personales - lag_1_mprestamos_personales as delta_1_mprestamos_personales
  ,lag(cprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_prendarios  
  ,lag(cprestamos_prendarios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_prendarios 
  ,lag(cprestamos_prendarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_prendarios  
  ,cprestamos_prendarios - lag_1_cprestamos_prendarios as delta_1_cprestamos_prendarios
  ,lag(mprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_prendarios  
  ,lag(mprestamos_prendarios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_prendarios 
  ,lag(mprestamos_prendarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_prendarios  
  ,mprestamos_prendarios - lag_1_mprestamos_prendarios as delta_1_mprestamos_prendarios
  ,lag(cprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_hipotecarios  
  ,lag(cprestamos_hipotecarios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_hipotecarios 
  ,lag(cprestamos_hipotecarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_hipotecarios  
  ,cprestamos_hipotecarios - lag_1_cprestamos_hipotecarios as delta_1_cprestamos_hipotecarios
  ,lag(mprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_hipotecarios  
  ,lag(mprestamos_hipotecarios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_hipotecarios 
  ,lag(mprestamos_hipotecarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_hipotecarios  
  ,mprestamos_hipotecarios - lag_1_mprestamos_hipotecarios as delta_1_mprestamos_hipotecarios
  ,lag(cplazo_fijo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cplazo_fijo  
  ,lag(cplazo_fijo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cplazo_fijo 
  ,lag(cplazo_fijo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cplazo_fijo  
  ,cplazo_fijo - lag_1_cplazo_fijo as delta_1_cplazo_fijo
  ,lag(mplazo_fijo_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_dolares  
  ,lag(mplazo_fijo_dolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mplazo_fijo_dolares 
  ,lag(mplazo_fijo_dolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mplazo_fijo_dolares  
  ,mplazo_fijo_dolares - lag_1_mplazo_fijo_dolares as delta_1_mplazo_fijo_dolares
  ,lag(mplazo_fijo_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_pesos  
  ,lag(mplazo_fijo_pesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mplazo_fijo_pesos 
  ,lag(mplazo_fijo_pesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mplazo_fijo_pesos  
  ,mplazo_fijo_pesos - lag_1_mplazo_fijo_pesos as delta_1_mplazo_fijo_pesos
  ,lag(cinversion1, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cinversion1  
  ,lag(cinversion1, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cinversion1 
  ,lag(cinversion1, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cinversion1  
  ,cinversion1 - lag_1_cinversion1 as delta_1_cinversion1
  ,lag(minversion1_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_pesos  
  ,lag(minversion1_pesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_minversion1_pesos 
  ,lag(minversion1_pesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_minversion1_pesos  
  ,minversion1_pesos - lag_1_minversion1_pesos as delta_1_minversion1_pesos
  ,lag(minversion1_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_dolares  
  ,lag(minversion1_dolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_minversion1_dolares 
  ,lag(minversion1_dolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_minversion1_dolares  
  ,minversion1_dolares - lag_1_minversion1_dolares as delta_1_minversion1_dolares
  ,lag(cinversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cinversion2  
  ,lag(cinversion2, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cinversion2 
  ,lag(cinversion2, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cinversion2  
  ,cinversion2 - lag_1_cinversion2 as delta_1_cinversion2
  ,lag(minversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion2  
  ,lag(minversion2, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_minversion2 
  ,lag(minversion2, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_minversion2  
  ,minversion2 - lag_1_minversion2 as delta_1_minversion2
  ,lag(cseguro_vida, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_vida  
  ,lag(cseguro_vida, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cseguro_vida 
  ,lag(cseguro_vida, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_vida  
  ,cseguro_vida - lag_1_cseguro_vida as delta_1_cseguro_vida
  ,lag(cseguro_auto, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_auto  
  ,lag(cseguro_auto, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cseguro_auto 
  ,lag(cseguro_auto, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_auto  
  ,cseguro_auto - lag_1_cseguro_auto as delta_1_cseguro_auto
  ,lag(cseguro_vivienda, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_vivienda  
  ,lag(cseguro_vivienda, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cseguro_vivienda 
  ,lag(cseguro_vivienda, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_vivienda  
  ,cseguro_vivienda - lag_1_cseguro_vivienda as delta_1_cseguro_vivienda
  ,lag(cseguro_accidentes_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_accidentes_personales  
  ,lag(cseguro_accidentes_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cseguro_accidentes_personales 
  ,lag(cseguro_accidentes_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_accidentes_personales  
  ,cseguro_accidentes_personales - lag_1_cseguro_accidentes_personales as delta_1_cseguro_accidentes_personales
  ,lag(ccaja_seguridad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccaja_seguridad  
  ,lag(ccaja_seguridad, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccaja_seguridad 
  ,lag(ccaja_seguridad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccaja_seguridad  
  ,ccaja_seguridad - lag_1_ccaja_seguridad as delta_1_ccaja_seguridad
  ,lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx  
  ,lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx 
  ,lag(cpayroll_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpayroll_trx  
  ,cpayroll_trx - lag_1_cpayroll_trx as delta_1_cpayroll_trx
  ,lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll  
  ,lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll 
  ,lag(mpayroll, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpayroll  
  ,mpayroll - lag_1_mpayroll as delta_1_mpayroll
  ,lag(mpayroll2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2  
  ,lag(mpayroll2, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll2 
  ,lag(mpayroll2, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpayroll2  
  ,mpayroll2 - lag_1_mpayroll2 as delta_1_mpayroll2
  ,lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx  
  ,lag(cpayroll2_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll2_trx 
  ,lag(cpayroll2_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpayroll2_trx  
  ,cpayroll2_trx - lag_1_cpayroll2_trx as delta_1_cpayroll2_trx
  ,lag(ccuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccuenta_debitos_automaticos  
  ,lag(ccuenta_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccuenta_debitos_automaticos 
  ,lag(ccuenta_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccuenta_debitos_automaticos  
  ,ccuenta_debitos_automaticos - lag_1_ccuenta_debitos_automaticos as delta_1_ccuenta_debitos_automaticos
  ,lag(mcuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_debitos_automaticos  
  ,lag(mcuenta_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_debitos_automaticos 
  ,lag(mcuenta_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_debitos_automaticos  
  ,mcuenta_debitos_automaticos - lag_1_mcuenta_debitos_automaticos as delta_1_mcuenta_debitos_automaticos
  ,lag(ctarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_debitos_automaticos  
  ,lag(ctarjeta_visa_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa_debitos_automaticos 
  ,lag(ctarjeta_visa_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_debitos_automaticos  
  ,ctarjeta_visa_debitos_automaticos - lag_1_ctarjeta_visa_debitos_automaticos as delta_1_ctarjeta_visa_debitos_automaticos
  ,lag(mttarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_visa_debitos_automaticos  
  ,lag(mttarjeta_visa_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mttarjeta_visa_debitos_automaticos 
  ,lag(mttarjeta_visa_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mttarjeta_visa_debitos_automaticos  
  ,mttarjeta_visa_debitos_automaticos - lag_1_mttarjeta_visa_debitos_automaticos as delta_1_mttarjeta_visa_debitos_automaticos
  ,lag(ctarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_debitos_automaticos  
  ,lag(ctarjeta_master_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_master_debitos_automaticos 
  ,lag(ctarjeta_master_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master_debitos_automaticos  
  ,ctarjeta_master_debitos_automaticos - lag_1_ctarjeta_master_debitos_automaticos as delta_1_ctarjeta_master_debitos_automaticos
  ,lag(mttarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_master_debitos_automaticos  
  ,lag(mttarjeta_master_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mttarjeta_master_debitos_automaticos 
  ,lag(mttarjeta_master_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mttarjeta_master_debitos_automaticos  
  ,mttarjeta_master_debitos_automaticos - lag_1_mttarjeta_master_debitos_automaticos as delta_1_mttarjeta_master_debitos_automaticos
  ,lag(cpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpagodeservicios  
  ,lag(cpagodeservicios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpagodeservicios 
  ,lag(cpagodeservicios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpagodeservicios  
  ,cpagodeservicios - lag_1_cpagodeservicios as delta_1_cpagodeservicios
  ,lag(mpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagodeservicios  
  ,lag(mpagodeservicios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpagodeservicios 
  ,lag(mpagodeservicios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpagodeservicios  
  ,mpagodeservicios - lag_1_mpagodeservicios as delta_1_mpagodeservicios
  ,lag(cpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpagomiscuentas  
  ,lag(cpagomiscuentas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpagomiscuentas 
  ,lag(cpagomiscuentas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpagomiscuentas  
  ,cpagomiscuentas - lag_1_cpagomiscuentas as delta_1_cpagomiscuentas
  ,lag(mpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagomiscuentas  
  ,lag(mpagomiscuentas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpagomiscuentas 
  ,lag(mpagomiscuentas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpagomiscuentas  
  ,mpagomiscuentas - lag_1_mpagomiscuentas as delta_1_mpagomiscuentas
  ,lag(ccajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajeros_propios_descuentos  
  ,lag(ccajeros_propios_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajeros_propios_descuentos 
  ,lag(ccajeros_propios_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajeros_propios_descuentos  
  ,ccajeros_propios_descuentos - lag_1_ccajeros_propios_descuentos as delta_1_ccajeros_propios_descuentos
  ,lag(mcajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcajeros_propios_descuentos  
  ,lag(mcajeros_propios_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcajeros_propios_descuentos 
  ,lag(mcajeros_propios_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcajeros_propios_descuentos  
  ,mcajeros_propios_descuentos - lag_1_mcajeros_propios_descuentos as delta_1_mcajeros_propios_descuentos
  ,lag(ctarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_descuentos  
  ,lag(ctarjeta_visa_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa_descuentos 
  ,lag(ctarjeta_visa_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_descuentos  
  ,ctarjeta_visa_descuentos - lag_1_ctarjeta_visa_descuentos as delta_1_ctarjeta_visa_descuentos
  ,lag(mtarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_descuentos  
  ,lag(mtarjeta_visa_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_visa_descuentos 
  ,lag(mtarjeta_visa_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_visa_descuentos  
  ,mtarjeta_visa_descuentos - lag_1_mtarjeta_visa_descuentos as delta_1_mtarjeta_visa_descuentos
  ,lag(ctarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_descuentos  
  ,lag(ctarjeta_master_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_master_descuentos 
  ,lag(ctarjeta_master_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master_descuentos  
  ,ctarjeta_master_descuentos - lag_1_ctarjeta_master_descuentos as delta_1_ctarjeta_master_descuentos
  ,lag(mtarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_descuentos  
  ,lag(mtarjeta_master_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_master_descuentos 
  ,lag(mtarjeta_master_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_master_descuentos  
  ,mtarjeta_master_descuentos - lag_1_mtarjeta_master_descuentos as delta_1_mtarjeta_master_descuentos
  ,lag(ccomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_mantenimiento  
  ,lag(ccomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccomisiones_mantenimiento 
  ,lag(ccomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccomisiones_mantenimiento  
  ,ccomisiones_mantenimiento - lag_1_ccomisiones_mantenimiento as delta_1_ccomisiones_mantenimiento
  ,lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento  
  ,lag(mcomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_mantenimiento 
  ,lag(mcomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones_mantenimiento  
  ,mcomisiones_mantenimiento - lag_1_mcomisiones_mantenimiento as delta_1_mcomisiones_mantenimiento
  ,lag(ccomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_otras  
  ,lag(ccomisiones_otras, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccomisiones_otras 
  ,lag(ccomisiones_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccomisiones_otras  
  ,ccomisiones_otras - lag_1_ccomisiones_otras as delta_1_ccomisiones_otras
  ,lag(mcomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_otras  
  ,lag(mcomisiones_otras, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_otras 
  ,lag(mcomisiones_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones_otras  
  ,mcomisiones_otras - lag_1_mcomisiones_otras as delta_1_mcomisiones_otras
  ,lag(cforex, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex  
  ,lag(cforex, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cforex 
  ,lag(cforex, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cforex  
  ,cforex - lag_1_cforex as delta_1_cforex
  ,lag(cforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex_buy  
  ,lag(cforex_buy, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cforex_buy 
  ,lag(cforex_buy, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cforex_buy  
  ,cforex_buy - lag_1_cforex_buy as delta_1_cforex_buy
  ,lag(mforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_buy  
  ,lag(mforex_buy, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mforex_buy 
  ,lag(mforex_buy, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mforex_buy  
  ,mforex_buy - lag_1_mforex_buy as delta_1_mforex_buy
  ,lag(cforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex_sell  
  ,lag(cforex_sell, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cforex_sell 
  ,lag(cforex_sell, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cforex_sell  
  ,cforex_sell - lag_1_cforex_sell as delta_1_cforex_sell
  ,lag(mforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_sell  
  ,lag(mforex_sell, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mforex_sell 
  ,lag(mforex_sell, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mforex_sell  
  ,mforex_sell - lag_1_mforex_sell as delta_1_mforex_sell
  ,lag(ctransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctransferencias_recibidas  
  ,lag(ctransferencias_recibidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctransferencias_recibidas 
  ,lag(ctransferencias_recibidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctransferencias_recibidas  
  ,ctransferencias_recibidas - lag_1_ctransferencias_recibidas as delta_1_ctransferencias_recibidas
  ,lag(mtransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_recibidas  
  ,lag(mtransferencias_recibidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtransferencias_recibidas 
  ,lag(mtransferencias_recibidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtransferencias_recibidas  
  ,mtransferencias_recibidas - lag_1_mtransferencias_recibidas as delta_1_mtransferencias_recibidas
  ,lag(ctransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctransferencias_emitidas  
  ,lag(ctransferencias_emitidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctransferencias_emitidas 
  ,lag(ctransferencias_emitidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctransferencias_emitidas  
  ,ctransferencias_emitidas - lag_1_ctransferencias_emitidas as delta_1_ctransferencias_emitidas
  ,lag(mtransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_emitidas  
  ,lag(mtransferencias_emitidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtransferencias_emitidas 
  ,lag(mtransferencias_emitidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtransferencias_emitidas  
  ,mtransferencias_emitidas - lag_1_mtransferencias_emitidas as delta_1_mtransferencias_emitidas
  ,lag(cextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cextraccion_autoservicio  
  ,lag(cextraccion_autoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cextraccion_autoservicio 
  ,lag(cextraccion_autoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cextraccion_autoservicio  
  ,cextraccion_autoservicio - lag_1_cextraccion_autoservicio as delta_1_cextraccion_autoservicio
  ,lag(mextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mextraccion_autoservicio  
  ,lag(mextraccion_autoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mextraccion_autoservicio 
  ,lag(mextraccion_autoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mextraccion_autoservicio  
  ,mextraccion_autoservicio - lag_1_mextraccion_autoservicio as delta_1_mextraccion_autoservicio
  ,lag(ccheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_depositados  
  ,lag(ccheques_depositados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccheques_depositados 
  ,lag(ccheques_depositados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_depositados  
  ,ccheques_depositados - lag_1_ccheques_depositados as delta_1_ccheques_depositados
  ,lag(mcheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados  
  ,lag(mcheques_depositados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcheques_depositados 
  ,lag(mcheques_depositados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_depositados  
  ,mcheques_depositados - lag_1_mcheques_depositados as delta_1_mcheques_depositados
  ,lag(ccheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_emitidos  
  ,lag(ccheques_emitidos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccheques_emitidos 
  ,lag(ccheques_emitidos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_emitidos  
  ,ccheques_emitidos - lag_1_ccheques_emitidos as delta_1_ccheques_emitidos
  ,lag(mcheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos  
  ,lag(mcheques_emitidos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcheques_emitidos 
  ,lag(mcheques_emitidos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_emitidos  
  ,mcheques_emitidos - lag_1_mcheques_emitidos as delta_1_mcheques_emitidos
  ,lag(ccheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_depositados_rechazados  
  ,lag(ccheques_depositados_rechazados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccheques_depositados_rechazados 
  ,lag(ccheques_depositados_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_depositados_rechazados  
  ,ccheques_depositados_rechazados - lag_1_ccheques_depositados_rechazados as delta_1_ccheques_depositados_rechazados
  ,lag(mcheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados_rechazados  
  ,lag(mcheques_depositados_rechazados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcheques_depositados_rechazados 
  ,lag(mcheques_depositados_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_depositados_rechazados  
  ,mcheques_depositados_rechazados - lag_1_mcheques_depositados_rechazados as delta_1_mcheques_depositados_rechazados
  ,lag(ccheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_emitidos_rechazados  
  ,lag(ccheques_emitidos_rechazados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccheques_emitidos_rechazados 
  ,lag(ccheques_emitidos_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_emitidos_rechazados  
  ,ccheques_emitidos_rechazados - lag_1_ccheques_emitidos_rechazados as delta_1_ccheques_emitidos_rechazados
  ,lag(mcheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos_rechazados  
  ,lag(mcheques_emitidos_rechazados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcheques_emitidos_rechazados 
  ,lag(mcheques_emitidos_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_emitidos_rechazados  
  ,mcheques_emitidos_rechazados - lag_1_mcheques_emitidos_rechazados as delta_1_mcheques_emitidos_rechazados
  ,lag(tcallcenter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tcallcenter  
  ,lag(tcallcenter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_tcallcenter 
  ,lag(tcallcenter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_tcallcenter  
  ,tcallcenter - lag_1_tcallcenter as delta_1_tcallcenter
  ,lag(ccallcenter_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccallcenter_transacciones  
  ,lag(ccallcenter_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccallcenter_transacciones 
  ,lag(ccallcenter_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccallcenter_transacciones  
  ,ccallcenter_transacciones - lag_1_ccallcenter_transacciones as delta_1_ccallcenter_transacciones
  ,lag(thomebanking, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_thomebanking  
  ,lag(thomebanking, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_thomebanking 
  ,lag(thomebanking, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_thomebanking  
  ,thomebanking - lag_1_thomebanking as delta_1_thomebanking
  ,lag(chomebanking_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_chomebanking_transacciones  
  ,lag(chomebanking_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_chomebanking_transacciones 
  ,lag(chomebanking_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_chomebanking_transacciones  
  ,chomebanking_transacciones - lag_1_chomebanking_transacciones as delta_1_chomebanking_transacciones
  ,lag(ccajas_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_transacciones  
  ,lag(ccajas_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_transacciones 
  ,lag(ccajas_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_transacciones  
  ,ccajas_transacciones - lag_1_ccajas_transacciones as delta_1_ccajas_transacciones
  ,lag(ccajas_consultas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_consultas  
  ,lag(ccajas_consultas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_consultas 
  ,lag(ccajas_consultas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_consultas  
  ,ccajas_consultas - lag_1_ccajas_consultas as delta_1_ccajas_consultas
  ,lag(ccajas_depositos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_depositos  
  ,lag(ccajas_depositos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_depositos 
  ,lag(ccajas_depositos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_depositos  
  ,ccajas_depositos - lag_1_ccajas_depositos as delta_1_ccajas_depositos
  ,lag(ccajas_extracciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_extracciones  
  ,lag(ccajas_extracciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_extracciones 
  ,lag(ccajas_extracciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_extracciones  
  ,ccajas_extracciones - lag_1_ccajas_extracciones as delta_1_ccajas_extracciones
  ,lag(ccajas_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_otras  
  ,lag(ccajas_otras, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_otras 
  ,lag(ccajas_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_otras  
  ,ccajas_otras - lag_1_ccajas_otras as delta_1_ccajas_otras
  ,lag(catm_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_catm_trx  
  ,lag(catm_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_catm_trx 
  ,lag(catm_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_catm_trx  
  ,catm_trx - lag_1_catm_trx as delta_1_catm_trx
  ,lag(matm, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm  
  ,lag(matm, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_matm 
  ,lag(matm, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_matm  
  ,matm - lag_1_matm as delta_1_matm
  ,lag(catm_trx_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_catm_trx_other  
  ,lag(catm_trx_other, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_catm_trx_other 
  ,lag(catm_trx_other, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_catm_trx_other  
  ,catm_trx_other - lag_1_catm_trx_other as delta_1_catm_trx_other
  ,lag(matm_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm_other  
  ,lag(matm_other, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_matm_other 
  ,lag(matm_other, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_matm_other  
  ,matm_other - lag_1_matm_other as delta_1_matm_other
  ,lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter  
  ,lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter 
  ,lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter  
  ,ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  ,lag(Master_delinquency, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_delinquency  
  ,lag(Master_delinquency, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_delinquency 
  ,lag(Master_delinquency, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_delinquency  
  ,Master_delinquency - lag_1_Master_delinquency as delta_1_Master_delinquency
  ,lag(Master_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_status  
  ,lag(Master_status, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_status 
  ,lag(Master_status, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_status  
  ,Master_status - lag_1_Master_status as delta_1_Master_status
  ,lag(Master_mfinanciacion_limite, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mfinanciacion_limite  
  ,lag(Master_mfinanciacion_limite, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mfinanciacion_limite 
  ,lag(Master_mfinanciacion_limite, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mfinanciacion_limite  
  ,Master_mfinanciacion_limite - lag_1_Master_mfinanciacion_limite as delta_1_Master_mfinanciacion_limite
  ,lag(Master_Fvencimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_Fvencimiento  
  ,lag(Master_Fvencimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_Fvencimiento 
  ,lag(Master_Fvencimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_Fvencimiento  
  ,Master_Fvencimiento - lag_1_Master_Fvencimiento as delta_1_Master_Fvencimiento
  ,lag(Master_Finiciomora, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_Finiciomora  
  ,lag(Master_Finiciomora, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_Finiciomora 
  ,lag(Master_Finiciomora, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_Finiciomora  
  ,Master_Finiciomora - lag_1_Master_Finiciomora as delta_1_Master_Finiciomora
  ,lag(Master_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldototal  
  ,lag(Master_msaldototal, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_msaldototal 
  ,lag(Master_msaldototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_msaldototal  
  ,Master_msaldototal - lag_1_Master_msaldototal as delta_1_Master_msaldototal
  ,lag(Master_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldopesos  
  ,lag(Master_msaldopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_msaldopesos 
  ,lag(Master_msaldopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_msaldopesos  
  ,Master_msaldopesos - lag_1_Master_msaldopesos as delta_1_Master_msaldopesos
  ,lag(Master_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldodolares  
  ,lag(Master_msaldodolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_msaldodolares 
  ,lag(Master_msaldodolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_msaldodolares  
  ,Master_msaldodolares - lag_1_Master_msaldodolares as delta_1_Master_msaldodolares
  ,lag(Master_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumospesos  
  ,lag(Master_mconsumospesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mconsumospesos 
  ,lag(Master_mconsumospesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mconsumospesos  
  ,Master_mconsumospesos - lag_1_Master_mconsumospesos as delta_1_Master_mconsumospesos
  ,lag(Master_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumosdolares  
  ,lag(Master_mconsumosdolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mconsumosdolares 
  ,lag(Master_mconsumosdolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mconsumosdolares  
  ,Master_mconsumosdolares - lag_1_Master_mconsumosdolares as delta_1_Master_mconsumosdolares
  ,lag(Master_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mlimitecompra  
  ,lag(Master_mlimitecompra, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mlimitecompra 
  ,lag(Master_mlimitecompra, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mlimitecompra  
  ,Master_mlimitecompra - lag_1_Master_mlimitecompra as delta_1_Master_mlimitecompra
  ,lag(Master_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantopesos  
  ,lag(Master_madelantopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_madelantopesos 
  ,lag(Master_madelantopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_madelantopesos  
  ,Master_madelantopesos - lag_1_Master_madelantopesos as delta_1_Master_madelantopesos
  ,lag(Master_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantodolares  
  ,lag(Master_madelantodolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_madelantodolares 
  ,lag(Master_madelantodolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_madelantodolares  
  ,Master_madelantodolares - lag_1_Master_madelantodolares as delta_1_Master_madelantodolares
  ,lag(Master_fultimo_cierre, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_fultimo_cierre  
  ,lag(Master_fultimo_cierre, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_fultimo_cierre 
  ,lag(Master_fultimo_cierre, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_fultimo_cierre  
  ,Master_fultimo_cierre - lag_1_Master_fultimo_cierre as delta_1_Master_fultimo_cierre
  ,lag(Master_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagado  
  ,lag(Master_mpagado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mpagado 
  ,lag(Master_mpagado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagado  
  ,Master_mpagado - lag_1_Master_mpagado as delta_1_Master_mpagado
  ,lag(Master_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagospesos  
  ,lag(Master_mpagospesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mpagospesos 
  ,lag(Master_mpagospesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagospesos  
  ,Master_mpagospesos - lag_1_Master_mpagospesos as delta_1_Master_mpagospesos
  ,lag(Master_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagosdolares  
  ,lag(Master_mpagosdolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mpagosdolares 
  ,lag(Master_mpagosdolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagosdolares  
  ,Master_mpagosdolares - lag_1_Master_mpagosdolares as delta_1_Master_mpagosdolares
  ,lag(Master_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_fechaalta  
  ,lag(Master_fechaalta, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_fechaalta 
  ,lag(Master_fechaalta, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_fechaalta  
  ,Master_fechaalta - lag_1_Master_fechaalta as delta_1_Master_fechaalta
  ,lag(Master_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumototal  
  ,lag(Master_mconsumototal, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mconsumototal 
  ,lag(Master_mconsumototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mconsumototal  
  ,Master_mconsumototal - lag_1_Master_mconsumototal as delta_1_Master_mconsumototal
  ,lag(Master_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cconsumos  
  ,lag(Master_cconsumos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_cconsumos 
  ,lag(Master_cconsumos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_cconsumos  
  ,Master_cconsumos - lag_1_Master_cconsumos as delta_1_Master_cconsumos
  ,lag(Master_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cadelantosefectivo  
  ,lag(Master_cadelantosefectivo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_cadelantosefectivo 
  ,lag(Master_cadelantosefectivo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_cadelantosefectivo  
  ,Master_cadelantosefectivo - lag_1_Master_cadelantosefectivo as delta_1_Master_cadelantosefectivo
  ,lag(Master_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagominimo  
  ,lag(Master_mpagominimo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mpagominimo 
  ,lag(Master_mpagominimo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagominimo  
  ,Master_mpagominimo - lag_1_Master_mpagominimo as delta_1_Master_mpagominimo
  ,lag(Visa_delinquency, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_delinquency  
  ,lag(Visa_delinquency, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_delinquency 
  ,lag(Visa_delinquency, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_delinquency  
  ,Visa_delinquency - lag_1_Visa_delinquency as delta_1_Visa_delinquency
  ,lag(Visa_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_status  
  ,lag(Visa_status, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_status 
  ,lag(Visa_status, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_status  
  ,Visa_status - lag_1_Visa_status as delta_1_Visa_status
  ,lag(Visa_mfinanciacion_limite, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mfinanciacion_limite  
  ,lag(Visa_mfinanciacion_limite, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mfinanciacion_limite 
  ,lag(Visa_mfinanciacion_limite, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mfinanciacion_limite  
  ,Visa_mfinanciacion_limite - lag_1_Visa_mfinanciacion_limite as delta_1_Visa_mfinanciacion_limite
  ,lag(Visa_Fvencimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Fvencimiento  
  ,lag(Visa_Fvencimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_Fvencimiento 
  ,lag(Visa_Fvencimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Fvencimiento  
  ,Visa_Fvencimiento - lag_1_Visa_Fvencimiento as delta_1_Visa_Fvencimiento
  ,lag(Visa_Finiciomora, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Finiciomora  
  ,lag(Visa_Finiciomora, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_Finiciomora 
  ,lag(Visa_Finiciomora, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Finiciomora  
  ,Visa_Finiciomora - lag_1_Visa_Finiciomora as delta_1_Visa_Finiciomora
  ,lag(Visa_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldototal  
  ,lag(Visa_msaldototal, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_msaldototal 
  ,lag(Visa_msaldototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldototal  
  ,Visa_msaldototal - lag_1_Visa_msaldototal as delta_1_Visa_msaldototal
  ,lag(Visa_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldopesos  
  ,lag(Visa_msaldopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_msaldopesos 
  ,lag(Visa_msaldopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldopesos  
  ,Visa_msaldopesos - lag_1_Visa_msaldopesos as delta_1_Visa_msaldopesos
  ,lag(Visa_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldodolares  
  ,lag(Visa_msaldodolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_msaldodolares 
  ,lag(Visa_msaldodolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldodolares  
  ,Visa_msaldodolares - lag_1_Visa_msaldodolares as delta_1_Visa_msaldodolares
  ,lag(Visa_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumospesos  
  ,lag(Visa_mconsumospesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mconsumospesos 
  ,lag(Visa_mconsumospesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mconsumospesos  
  ,Visa_mconsumospesos - lag_1_Visa_mconsumospesos as delta_1_Visa_mconsumospesos
  ,lag(Visa_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumosdolares  
  ,lag(Visa_mconsumosdolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mconsumosdolares 
  ,lag(Visa_mconsumosdolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mconsumosdolares  
  ,Visa_mconsumosdolares - lag_1_Visa_mconsumosdolares as delta_1_Visa_mconsumosdolares
  ,lag(Visa_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mlimitecompra  
  ,lag(Visa_mlimitecompra, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mlimitecompra 
  ,lag(Visa_mlimitecompra, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mlimitecompra  
  ,Visa_mlimitecompra - lag_1_Visa_mlimitecompra as delta_1_Visa_mlimitecompra
  ,lag(Visa_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantopesos  
  ,lag(Visa_madelantopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_madelantopesos 
  ,lag(Visa_madelantopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_madelantopesos  
  ,Visa_madelantopesos - lag_1_Visa_madelantopesos as delta_1_Visa_madelantopesos
  ,lag(Visa_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantodolares  
  ,lag(Visa_madelantodolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_madelantodolares 
  ,lag(Visa_madelantodolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_madelantodolares  
  ,Visa_madelantodolares - lag_1_Visa_madelantodolares as delta_1_Visa_madelantodolares
  ,lag(Visa_fultimo_cierre, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_fultimo_cierre  
  ,lag(Visa_fultimo_cierre, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_fultimo_cierre 
  ,lag(Visa_fultimo_cierre, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_fultimo_cierre  
  ,Visa_fultimo_cierre - lag_1_Visa_fultimo_cierre as delta_1_Visa_fultimo_cierre
  ,lag(Visa_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagado  
  ,lag(Visa_mpagado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagado 
  ,lag(Visa_mpagado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagado  
  ,Visa_mpagado - lag_1_Visa_mpagado as delta_1_Visa_mpagado
  ,lag(Visa_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagospesos  
  ,lag(Visa_mpagospesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagospesos 
  ,lag(Visa_mpagospesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagospesos  
  ,Visa_mpagospesos - lag_1_Visa_mpagospesos as delta_1_Visa_mpagospesos
  ,lag(Visa_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagosdolares  
  ,lag(Visa_mpagosdolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagosdolares 
  ,lag(Visa_mpagosdolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagosdolares  
  ,Visa_mpagosdolares - lag_1_Visa_mpagosdolares as delta_1_Visa_mpagosdolares
  ,lag(Visa_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_fechaalta  
  ,lag(Visa_fechaalta, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_fechaalta 
  ,lag(Visa_fechaalta, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_fechaalta  
  ,Visa_fechaalta - lag_1_Visa_fechaalta as delta_1_Visa_fechaalta
  ,lag(Visa_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumototal  
  ,lag(Visa_mconsumototal, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mconsumototal 
  ,lag(Visa_mconsumototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mconsumototal  
  ,Visa_mconsumototal - lag_1_Visa_mconsumototal as delta_1_Visa_mconsumototal
  ,lag(Visa_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cconsumos  
  ,lag(Visa_cconsumos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_cconsumos 
  ,lag(Visa_cconsumos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_cconsumos  
  ,Visa_cconsumos - lag_1_Visa_cconsumos as delta_1_Visa_cconsumos
  ,lag(Visa_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cadelantosefectivo  
  ,lag(Visa_cadelantosefectivo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_cadelantosefectivo 
  ,lag(Visa_cadelantosefectivo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_cadelantosefectivo  
  ,Visa_cadelantosefectivo - lag_1_Visa_cadelantosefectivo as delta_1_Visa_cadelantosefectivo
  ,lag(Visa_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagominimo  
  ,lag(Visa_mpagominimo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagominimo 
  ,lag(Visa_mpagominimo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagominimo  
  ,Visa_mpagominimo - lag_1_Visa_mpagominimo as delta_1_Visa_mpagominimo
from competencia_02

Empty DataFrame
Columns: [Success]
Index: []

### Promedio movil: 

In [10]:
# Genero la query para cada campo
q_avg = ""
for campo in campos:
  q_avg += f"\n ,avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_{campo}"
#print(q_avg)

In [12]:
with open("C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/q_avg.txt", "w") as archivo:
    archivo.write(q_avg)

In [13]:
%%sql
create or replace table competencia_02_avg as
select
  numero_de_cliente
  ,foto_mes
  
 ,avg(active_quarter) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_active_quarter
 ,avg(cliente_vip) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cliente_vip
 ,avg(internet) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_internet
 ,avg(cliente_edad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cliente_edad
 ,avg(cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cliente_antiguedad
 ,avg(mrentabilidad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mrentabilidad
 ,avg(mrentabilidad_annual) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mrentabilidad_annual
 ,avg(mcomisiones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcomisiones
 ,avg(mactivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mactivos_margen
 ,avg(mpasivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mpasivos_margen
 ,avg(cproductos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cproductos
 ,avg(tcuentas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_tcuentas
 ,avg(ccuenta_corriente) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccuenta_corriente
 ,avg(mcuenta_corriente_adicional) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcuenta_corriente_adicional
 ,avg(mcuenta_corriente) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcuenta_corriente
 ,avg(ccaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccaja_ahorro
 ,avg(mcaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcaja_ahorro
 ,avg(mcaja_ahorro_adicional) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcaja_ahorro_adicional
 ,avg(mcaja_ahorro_dolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcaja_ahorro_dolares
 ,avg(cdescubierto_preacordado) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cdescubierto_preacordado
 ,avg(mcuentas_saldo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcuentas_saldo
 ,avg(ctarjeta_debito) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_debito
 ,avg(ctarjeta_debito_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_debito_transacciones
 ,avg(mautoservicio) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mautoservicio
 ,avg(ctarjeta_visa) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_visa
 ,avg(ctarjeta_visa_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_visa_transacciones
 ,avg(mtarjeta_visa_consumo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mtarjeta_visa_consumo
 ,avg(ctarjeta_master) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_master
 ,avg(ctarjeta_master_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_master_transacciones
 ,avg(mtarjeta_master_consumo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mtarjeta_master_consumo
 ,avg(cprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cprestamos_personales
 ,avg(mprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mprestamos_personales
 ,avg(cprestamos_prendarios) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cprestamos_prendarios
 ,avg(mprestamos_prendarios) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mprestamos_prendarios
 ,avg(cprestamos_hipotecarios) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cprestamos_hipotecarios
 ,avg(mprestamos_hipotecarios) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mprestamos_hipotecarios
 ,avg(cplazo_fijo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cplazo_fijo
 ,avg(mplazo_fijo_dolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mplazo_fijo_dolares
 ,avg(mplazo_fijo_pesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mplazo_fijo_pesos
 ,avg(cinversion1) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cinversion1
 ,avg(minversion1_pesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_minversion1_pesos
 ,avg(minversion1_dolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_minversion1_dolares
 ,avg(cinversion2) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cinversion2
 ,avg(minversion2) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_minversion2
 ,avg(cseguro_vida) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cseguro_vida
 ,avg(cseguro_auto) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cseguro_auto
 ,avg(cseguro_vivienda) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cseguro_vivienda
 ,avg(cseguro_accidentes_personales) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cseguro_accidentes_personales
 ,avg(ccaja_seguridad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccaja_seguridad
 ,avg(cpayroll_trx) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cpayroll_trx
 ,avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mpayroll
 ,avg(mpayroll2) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mpayroll2
 ,avg(cpayroll2_trx) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cpayroll2_trx
 ,avg(ccuenta_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccuenta_debitos_automaticos
 ,avg(mcuenta_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcuenta_debitos_automaticos
 ,avg(ctarjeta_visa_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_visa_debitos_automaticos
 ,avg(mttarjeta_visa_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mttarjeta_visa_debitos_automaticos
 ,avg(ctarjeta_master_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_master_debitos_automaticos
 ,avg(mttarjeta_master_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mttarjeta_master_debitos_automaticos
 ,avg(cpagodeservicios) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cpagodeservicios
 ,avg(mpagodeservicios) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mpagodeservicios
 ,avg(cpagomiscuentas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cpagomiscuentas
 ,avg(mpagomiscuentas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mpagomiscuentas
 ,avg(ccajeros_propios_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccajeros_propios_descuentos
 ,avg(mcajeros_propios_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcajeros_propios_descuentos
 ,avg(ctarjeta_visa_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_visa_descuentos
 ,avg(mtarjeta_visa_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mtarjeta_visa_descuentos
 ,avg(ctarjeta_master_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_master_descuentos
 ,avg(mtarjeta_master_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mtarjeta_master_descuentos
 ,avg(ccomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccomisiones_mantenimiento
 ,avg(mcomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcomisiones_mantenimiento
 ,avg(ccomisiones_otras) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccomisiones_otras
 ,avg(mcomisiones_otras) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcomisiones_otras
 ,avg(cforex) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cforex
 ,avg(cforex_buy) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cforex_buy
 ,avg(mforex_buy) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mforex_buy
 ,avg(cforex_sell) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cforex_sell
 ,avg(mforex_sell) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mforex_sell
 ,avg(ctransferencias_recibidas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctransferencias_recibidas
 ,avg(mtransferencias_recibidas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mtransferencias_recibidas
 ,avg(ctransferencias_emitidas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctransferencias_emitidas
 ,avg(mtransferencias_emitidas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mtransferencias_emitidas
 ,avg(cextraccion_autoservicio) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cextraccion_autoservicio
 ,avg(mextraccion_autoservicio) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mextraccion_autoservicio
 ,avg(ccheques_depositados) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccheques_depositados
 ,avg(mcheques_depositados) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcheques_depositados
 ,avg(ccheques_emitidos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccheques_emitidos
 ,avg(mcheques_emitidos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcheques_emitidos
 ,avg(ccheques_depositados_rechazados) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccheques_depositados_rechazados
 ,avg(mcheques_depositados_rechazados) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcheques_depositados_rechazados
 ,avg(ccheques_emitidos_rechazados) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccheques_emitidos_rechazados
 ,avg(mcheques_emitidos_rechazados) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcheques_emitidos_rechazados
 ,avg(tcallcenter) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_tcallcenter
 ,avg(ccallcenter_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccallcenter_transacciones
 ,avg(thomebanking) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_thomebanking
 ,avg(chomebanking_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_chomebanking_transacciones
 ,avg(ccajas_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccajas_transacciones
 ,avg(ccajas_consultas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccajas_consultas
 ,avg(ccajas_depositos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccajas_depositos
 ,avg(ccajas_extracciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccajas_extracciones
 ,avg(ccajas_otras) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccajas_otras
 ,avg(catm_trx) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_catm_trx
 ,avg(matm) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_matm
 ,avg(catm_trx_other) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_catm_trx_other
 ,avg(matm_other) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_matm_other
 ,avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctrx_quarter
 ,avg(Master_delinquency) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_delinquency
 ,avg(Master_status) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_status
 ,avg(Master_mfinanciacion_limite) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_mfinanciacion_limite
 ,avg(Master_Fvencimiento) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_Fvencimiento
 ,avg(Master_Finiciomora) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_Finiciomora
 ,avg(Master_msaldototal) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_msaldototal
 ,avg(Master_msaldopesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_msaldopesos
 ,avg(Master_msaldodolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_msaldodolares
 ,avg(Master_mconsumospesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_mconsumospesos
 ,avg(Master_mconsumosdolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_mconsumosdolares
 ,avg(Master_mlimitecompra) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_mlimitecompra
 ,avg(Master_madelantopesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_madelantopesos
 ,avg(Master_madelantodolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_madelantodolares
 ,avg(Master_fultimo_cierre) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_fultimo_cierre
 ,avg(Master_mpagado) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_mpagado
 ,avg(Master_mpagospesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_mpagospesos
 ,avg(Master_mpagosdolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_mpagosdolares
 ,avg(Master_fechaalta) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_fechaalta
 ,avg(Master_mconsumototal) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_mconsumototal
 ,avg(Master_cconsumos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_cconsumos
 ,avg(Master_cadelantosefectivo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_cadelantosefectivo
 ,avg(Master_mpagominimo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_mpagominimo
 ,avg(Visa_delinquency) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_delinquency
 ,avg(Visa_status) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_status
 ,avg(Visa_mfinanciacion_limite) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mfinanciacion_limite
 ,avg(Visa_Fvencimiento) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_Fvencimiento
 ,avg(Visa_Finiciomora) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_Finiciomora
 ,avg(Visa_msaldototal) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_msaldototal
 ,avg(Visa_msaldopesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_msaldopesos
 ,avg(Visa_msaldodolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_msaldodolares
 ,avg(Visa_mconsumospesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mconsumospesos
 ,avg(Visa_mconsumosdolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mconsumosdolares
 ,avg(Visa_mlimitecompra) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mlimitecompra
 ,avg(Visa_madelantopesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_madelantopesos
 ,avg(Visa_madelantodolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_madelantodolares
 ,avg(Visa_fultimo_cierre) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_fultimo_cierre
 ,avg(Visa_mpagado) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mpagado
 ,avg(Visa_mpagospesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mpagospesos
 ,avg(Visa_mpagosdolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mpagosdolares
 ,avg(Visa_fechaalta) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_fechaalta
 ,avg(Visa_mconsumototal) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mconsumototal
 ,avg(Visa_cconsumos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_cconsumos
 ,avg(Visa_cadelantosefectivo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_cadelantosefectivo
 ,avg(Visa_mpagominimo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mpagominimo
from competencia_02

Empty DataFrame
Columns: [Success]
Index: []

### TENDENCIA:

In [14]:
# Genero la query para cada campo
q_reg = ""
for campo in campos:
  q_reg += f"\n ,regr_slope({campo}, cliente_antiguedad) over ventana_6 as {campo}_slope_6"
#print(q_reg)

In [15]:
with open("C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/q_reg.txt", "w") as archivo:
    archivo.write(q_reg)

In [16]:
%%sql
create or replace table competencia_02_reg as
SELECT 
    numero_de_cliente
    ,foto_mes
    ,regr_slope(active_quarter, cliente_antiguedad) over ventana_6 as active_quarter_slope_6
    ,regr_slope(cliente_vip, cliente_antiguedad) over ventana_6 as cliente_vip_slope_6
    ,regr_slope(internet, cliente_antiguedad) over ventana_6 as internet_slope_6
    ,regr_slope(cliente_edad, cliente_antiguedad) over ventana_6 as cliente_edad_slope_6
    ,regr_slope(cliente_antiguedad, cliente_antiguedad) over ventana_6 as cliente_antiguedad_slope_6
    ,regr_slope(mrentabilidad, cliente_antiguedad) over ventana_6 as mrentabilidad_slope_6
    ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_6 as mrentabilidad_annual_slope_6
    ,regr_slope(mcomisiones, cliente_antiguedad) over ventana_6 as mcomisiones_slope_6
    ,regr_slope(mactivos_margen, cliente_antiguedad) over ventana_6 as mactivos_margen_slope_6
    ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as mpasivos_margen_slope_6
    ,regr_slope(cproductos, cliente_antiguedad) over ventana_6 as cproductos_slope_6
    ,regr_slope(tcuentas, cliente_antiguedad) over ventana_6 as tcuentas_slope_6
    ,regr_slope(ccuenta_corriente, cliente_antiguedad) over ventana_6 as ccuenta_corriente_slope_6
    ,regr_slope(mcuenta_corriente_adicional, cliente_antiguedad) over ventana_6 as mcuenta_corriente_adicional_slope_6
    ,regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_6 as mcuenta_corriente_slope_6
    ,regr_slope(ccaja_ahorro, cliente_antiguedad) over ventana_6 as ccaja_ahorro_slope_6
    ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as mcaja_ahorro_slope_6
    ,regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_6 as mcaja_ahorro_adicional_slope_6
    ,regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_6 as mcaja_ahorro_dolares_slope_6
    ,regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_6 as cdescubierto_preacordado_slope_6
    ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_6 as mcuentas_saldo_slope_6
    ,regr_slope(ctarjeta_debito, cliente_antiguedad) over ventana_6 as ctarjeta_debito_slope_6
    ,regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_debito_transacciones_slope_6
    ,regr_slope(mautoservicio, cliente_antiguedad) over ventana_6 as mautoservicio_slope_6
    ,regr_slope(ctarjeta_visa, cliente_antiguedad) over ventana_6 as ctarjeta_visa_slope_6
    ,regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_visa_transacciones_slope_6
    ,regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_6 as mtarjeta_visa_consumo_slope_6
    ,regr_slope(ctarjeta_master, cliente_antiguedad) over ventana_6 as ctarjeta_master_slope_6
    ,regr_slope(ctarjeta_master_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_master_transacciones_slope_6
    ,regr_slope(mtarjeta_master_consumo, cliente_antiguedad) over ventana_6 as mtarjeta_master_consumo_slope_6
    ,regr_slope(cprestamos_personales, cliente_antiguedad) over ventana_6 as cprestamos_personales_slope_6
    ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_6 as mprestamos_personales_slope_6
    ,regr_slope(cprestamos_prendarios, cliente_antiguedad) over ventana_6 as cprestamos_prendarios_slope_6
    ,regr_slope(mprestamos_prendarios, cliente_antiguedad) over ventana_6 as mprestamos_prendarios_slope_6
    ,regr_slope(cprestamos_hipotecarios, cliente_antiguedad) over ventana_6 as cprestamos_hipotecarios_slope_6
    ,regr_slope(mprestamos_hipotecarios, cliente_antiguedad) over ventana_6 as mprestamos_hipotecarios_slope_6
    ,regr_slope(cplazo_fijo, cliente_antiguedad) over ventana_6 as cplazo_fijo_slope_6
    ,regr_slope(mplazo_fijo_dolares, cliente_antiguedad) over ventana_6 as mplazo_fijo_dolares_slope_6
    ,regr_slope(mplazo_fijo_pesos, cliente_antiguedad) over ventana_6 as mplazo_fijo_pesos_slope_6
    ,regr_slope(cinversion1, cliente_antiguedad) over ventana_6 as cinversion1_slope_6
    ,regr_slope(minversion1_pesos, cliente_antiguedad) over ventana_6 as minversion1_pesos_slope_6
    ,regr_slope(minversion1_dolares, cliente_antiguedad) over ventana_6 as minversion1_dolares_slope_6
    ,regr_slope(cinversion2, cliente_antiguedad) over ventana_6 as cinversion2_slope_6
    ,regr_slope(minversion2, cliente_antiguedad) over ventana_6 as minversion2_slope_6
    ,regr_slope(cseguro_vida, cliente_antiguedad) over ventana_6 as cseguro_vida_slope_6
    ,regr_slope(cseguro_auto, cliente_antiguedad) over ventana_6 as cseguro_auto_slope_6
    ,regr_slope(cseguro_vivienda, cliente_antiguedad) over ventana_6 as cseguro_vivienda_slope_6
    ,regr_slope(cseguro_accidentes_personales, cliente_antiguedad) over ventana_6 as cseguro_accidentes_personales_slope_6
    ,regr_slope(ccaja_seguridad, cliente_antiguedad) over ventana_6 as ccaja_seguridad_slope_6
    ,regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_6 as cpayroll_trx_slope_6
    ,regr_slope(mpayroll, cliente_antiguedad) over ventana_6 as mpayroll_slope_6
    ,regr_slope(mpayroll2, cliente_antiguedad) over ventana_6 as mpayroll2_slope_6
    ,regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_6 as cpayroll2_trx_slope_6
    ,regr_slope(ccuenta_debitos_automaticos, cliente_antiguedad) over ventana_6 as ccuenta_debitos_automaticos_slope_6
    ,regr_slope(mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_6 as mcuenta_debitos_automaticos_slope_6
    ,regr_slope(ctarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctarjeta_visa_debitos_automaticos_slope_6
    ,regr_slope(mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_6 as mttarjeta_visa_debitos_automaticos_slope_6
    ,regr_slope(ctarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctarjeta_master_debitos_automaticos_slope_6
    ,regr_slope(mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_6 as mttarjeta_master_debitos_automaticos_slope_6
    ,regr_slope(cpagodeservicios, cliente_antiguedad) over ventana_6 as cpagodeservicios_slope_6
    ,regr_slope(mpagodeservicios, cliente_antiguedad) over ventana_6 as mpagodeservicios_slope_6
    ,regr_slope(cpagomiscuentas, cliente_antiguedad) over ventana_6 as cpagomiscuentas_slope_6
    ,regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_6 as mpagomiscuentas_slope_6
    ,regr_slope(ccajeros_propios_descuentos, cliente_antiguedad) over ventana_6 as ccajeros_propios_descuentos_slope_6
    ,regr_slope(mcajeros_propios_descuentos, cliente_antiguedad) over ventana_6 as mcajeros_propios_descuentos_slope_6
    ,regr_slope(ctarjeta_visa_descuentos, cliente_antiguedad) over ventana_6 as ctarjeta_visa_descuentos_slope_6
    ,regr_slope(mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_6 as mtarjeta_visa_descuentos_slope_6
    ,regr_slope(ctarjeta_master_descuentos, cliente_antiguedad) over ventana_6 as ctarjeta_master_descuentos_slope_6
    ,regr_slope(mtarjeta_master_descuentos, cliente_antiguedad) over ventana_6 as mtarjeta_master_descuentos_slope_6
    ,regr_slope(ccomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as ccomisiones_mantenimiento_slope_6
    ,regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as mcomisiones_mantenimiento_slope_6
    ,regr_slope(ccomisiones_otras, cliente_antiguedad) over ventana_6 as ccomisiones_otras_slope_6
    ,regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_6 as mcomisiones_otras_slope_6
    ,regr_slope(cforex, cliente_antiguedad) over ventana_6 as cforex_slope_6
    ,regr_slope(cforex_buy, cliente_antiguedad) over ventana_6 as cforex_buy_slope_6
    ,regr_slope(mforex_buy, cliente_antiguedad) over ventana_6 as mforex_buy_slope_6
    ,regr_slope(cforex_sell, cliente_antiguedad) over ventana_6 as cforex_sell_slope_6
    ,regr_slope(mforex_sell, cliente_antiguedad) over ventana_6 as mforex_sell_slope_6
    ,regr_slope(ctransferencias_recibidas, cliente_antiguedad) over ventana_6 as ctransferencias_recibidas_slope_6
    ,regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_6 as mtransferencias_recibidas_slope_6
    ,regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_6 as ctransferencias_emitidas_slope_6
    ,regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_6 as mtransferencias_emitidas_slope_6
    ,regr_slope(cextraccion_autoservicio, cliente_antiguedad) over ventana_6 as cextraccion_autoservicio_slope_6
    ,regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_6 as mextraccion_autoservicio_slope_6
    ,regr_slope(ccheques_depositados, cliente_antiguedad) over ventana_6 as ccheques_depositados_slope_6
    ,regr_slope(mcheques_depositados, cliente_antiguedad) over ventana_6 as mcheques_depositados_slope_6
    ,regr_slope(ccheques_emitidos, cliente_antiguedad) over ventana_6 as ccheques_emitidos_slope_6
    ,regr_slope(mcheques_emitidos, cliente_antiguedad) over ventana_6 as mcheques_emitidos_slope_6
    ,regr_slope(ccheques_depositados_rechazados, cliente_antiguedad) over ventana_6 as ccheques_depositados_rechazados_slope_6
    ,regr_slope(mcheques_depositados_rechazados, cliente_antiguedad) over ventana_6 as mcheques_depositados_rechazados_slope_6
    ,regr_slope(ccheques_emitidos_rechazados, cliente_antiguedad) over ventana_6 as ccheques_emitidos_rechazados_slope_6
    ,regr_slope(mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_6 as mcheques_emitidos_rechazados_slope_6
    ,regr_slope(tcallcenter, cliente_antiguedad) over ventana_6 as tcallcenter_slope_6
    ,regr_slope(ccallcenter_transacciones, cliente_antiguedad) over ventana_6 as ccallcenter_transacciones_slope_6
    ,regr_slope(thomebanking, cliente_antiguedad) over ventana_6 as thomebanking_slope_6
    ,regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_6 as chomebanking_transacciones_slope_6
    ,regr_slope(ccajas_transacciones, cliente_antiguedad) over ventana_6 as ccajas_transacciones_slope_6
    ,regr_slope(ccajas_consultas, cliente_antiguedad) over ventana_6 as ccajas_consultas_slope_6
    ,regr_slope(ccajas_depositos, cliente_antiguedad) over ventana_6 as ccajas_depositos_slope_6
    ,regr_slope(ccajas_extracciones, cliente_antiguedad) over ventana_6 as ccajas_extracciones_slope_6
    ,regr_slope(ccajas_otras, cliente_antiguedad) over ventana_6 as ccajas_otras_slope_6
    ,regr_slope(catm_trx, cliente_antiguedad) over ventana_6 as catm_trx_slope_6
    ,regr_slope(matm, cliente_antiguedad) over ventana_6 as matm_slope_6
    ,regr_slope(catm_trx_other, cliente_antiguedad) over ventana_6 as catm_trx_other_slope_6
    ,regr_slope(matm_other, cliente_antiguedad) over ventana_6 as matm_other_slope_6
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6
    ,regr_slope(Master_delinquency, cliente_antiguedad) over ventana_6 as Master_delinquency_slope_6
    ,regr_slope(Master_status, cliente_antiguedad) over ventana_6 as Master_status_slope_6
    ,regr_slope(Master_mfinanciacion_limite, cliente_antiguedad) over ventana_6 as Master_mfinanciacion_limite_slope_6
    ,regr_slope(Master_Fvencimiento, cliente_antiguedad) over ventana_6 as Master_Fvencimiento_slope_6
    ,regr_slope(Master_Finiciomora, cliente_antiguedad) over ventana_6 as Master_Finiciomora_slope_6
    ,regr_slope(Master_msaldototal, cliente_antiguedad) over ventana_6 as Master_msaldototal_slope_6
    ,regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana_6 as Master_msaldopesos_slope_6
    ,regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana_6 as Master_msaldodolares_slope_6
    ,regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana_6 as Master_mconsumospesos_slope_6
    ,regr_slope(Master_mconsumosdolares, cliente_antiguedad) over ventana_6 as Master_mconsumosdolares_slope_6
    ,regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana_6 as Master_mlimitecompra_slope_6
    ,regr_slope(Master_madelantopesos, cliente_antiguedad) over ventana_6 as Master_madelantopesos_slope_6
    ,regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana_6 as Master_madelantodolares_slope_6
    ,regr_slope(Master_fultimo_cierre, cliente_antiguedad) over ventana_6 as Master_fultimo_cierre_slope_6
    ,regr_slope(Master_mpagado, cliente_antiguedad) over ventana_6 as Master_mpagado_slope_6
    ,regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana_6 as Master_mpagospesos_slope_6
    ,regr_slope(Master_mpagosdolares, cliente_antiguedad) over ventana_6 as Master_mpagosdolares_slope_6
    ,regr_slope(Master_fechaalta, cliente_antiguedad) over ventana_6 as Master_fechaalta_slope_6
    ,regr_slope(Master_mconsumototal, cliente_antiguedad) over ventana_6 as Master_mconsumototal_slope_6
    ,regr_slope(Master_cconsumos, cliente_antiguedad) over ventana_6 as Master_cconsumos_slope_6
    ,regr_slope(Master_cadelantosefectivo, cliente_antiguedad) over ventana_6 as Master_cadelantosefectivo_slope_6
    ,regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana_6 as Master_mpagominimo_slope_6
    ,regr_slope(Visa_delinquency, cliente_antiguedad) over ventana_6 as Visa_delinquency_slope_6
    ,regr_slope(Visa_status, cliente_antiguedad) over ventana_6 as Visa_status_slope_6
    ,regr_slope(Visa_mfinanciacion_limite, cliente_antiguedad) over ventana_6 as Visa_mfinanciacion_limite_slope_6
    ,regr_slope(Visa_Fvencimiento, cliente_antiguedad) over ventana_6 as Visa_Fvencimiento_slope_6
    ,regr_slope(Visa_Finiciomora, cliente_antiguedad) over ventana_6 as Visa_Finiciomora_slope_6
    ,regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_6 as Visa_msaldototal_slope_6
    ,regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_6 as Visa_msaldopesos_slope_6
    ,regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana_6 as Visa_msaldodolares_slope_6
    ,regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana_6 as Visa_mconsumospesos_slope_6
    ,regr_slope(Visa_mconsumosdolares, cliente_antiguedad) over ventana_6 as Visa_mconsumosdolares_slope_6
    ,regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana_6 as Visa_mlimitecompra_slope_6
    ,regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_6 as Visa_madelantopesos_slope_6
    ,regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_6 as Visa_madelantodolares_slope_6
    ,regr_slope(Visa_fultimo_cierre, cliente_antiguedad) over ventana_6 as Visa_fultimo_cierre_slope_6
    ,regr_slope(Visa_mpagado, cliente_antiguedad) over ventana_6 as Visa_mpagado_slope_6
    ,regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana_6 as Visa_mpagospesos_slope_6
    ,regr_slope(Visa_mpagosdolares, cliente_antiguedad) over ventana_6 as Visa_mpagosdolares_slope_6
    ,regr_slope(Visa_fechaalta, cliente_antiguedad) over ventana_6 as Visa_fechaalta_slope_6
    ,regr_slope(Visa_mconsumototal, cliente_antiguedad) over ventana_6 as Visa_mconsumototal_slope_6
    ,regr_slope(Visa_cconsumos, cliente_antiguedad) over ventana_6 as Visa_cconsumos_slope_6
    ,regr_slope(Visa_cadelantosefectivo, cliente_antiguedad) over ventana_6 as Visa_cadelantosefectivo_slope_6
    ,regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_6 as Visa_mpagominimo_slope_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)

Empty DataFrame
Columns: [Success]
Index: []

# Union tabla final:

In [3]:
%%sql
SELECT * FROM competencia_02_lag_delta 
LIMIT 2

numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
0           29211377    201901               1            0         1   
1           29211377    201902               1            0         1   

   cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
0            52                 296        8128.67              29300.39   
1            52                 297        1779.59              28968.19   

   mcomisiones  ...  lag_3_Visa_cconsumos  delta_1_Visa_cconsumos  \
0      5092.45  ...                   NaN                     NaN   
1       307.68  ...                   NaN                     1.0   

   lag_1_Visa_cadelantosefectivo  lag_2_Visa_cadelantosefectivo  \
0                            NaN                            NaN   
1                            0.0                            NaN   

   lag_3_Visa_cadelantosefectivo  delta_1_Visa_cadelantosefectivo  \
0                            NaN                              NaN   
1                            NaN                              0.0   

   lag_1_Visa_mpagominimo  lag_2_Visa_mpagominimo  lag_3_Visa_mpagominimo  \
0                     NaN                     NaN                     NaN   
1                 1313.76                     NaN                     NaN   

   delta_1_Visa_mpagominimo  
0                       NaN  
1                    -93.84  

[2 rows x 752 columns]

In [4]:
%%sql
SELECT * FROM competencia_02_avg
LIMIT 2

numero_de_cliente  foto_mes  avg_6_active_quarter  avg_6_cliente_vip  \
0           29253623    201901                   1.0                0.0   
1           29253623    201902                   1.0                0.0   

   avg_6_internet  avg_6_cliente_edad  avg_6_cliente_antiguedad  \
0             1.0                56.0                     251.0   
1             1.0                56.0                     251.5   

   avg_6_mrentabilidad  avg_6_mrentabilidad_annual  avg_6_mcomisiones  ...  \
0              2132.11                   21043.470             259.19  ...   
1              2100.98                   21207.845             182.39  ...   

   avg_6_Visa_madelantodolares  avg_6_Visa_fultimo_cierre  avg_6_Visa_mpagado  \
0                          NaN                        NaN                 NaN   
1                          NaN                        NaN                 NaN   

   avg_6_Visa_mpagospesos  avg_6_Visa_mpagosdolares  avg_6_Visa_fechaalta  \
0                     NaN                       NaN                   NaN   
1                     NaN                       NaN                   NaN   

   avg_6_Visa_mconsumototal  avg_6_Visa_cconsumos  \
0                       NaN                   NaN   
1                       NaN                   NaN   

   avg_6_Visa_cadelantosefectivo  avg_6_Visa_mpagominimo  
0                            NaN                     NaN  
1                            NaN                     NaN  

[2 rows x 152 columns]

In [5]:
%%sql
SELECT * FROM competencia_02_reg
LIMIT 2

numero_de_cliente  foto_mes  active_quarter_slope_6  cliente_vip_slope_6  \
0           29188883    201901                     NaN                  NaN   
1           29188883    201902                     0.0                  0.0   

   internet_slope_6  cliente_edad_slope_6  cliente_antiguedad_slope_6  \
0               NaN                   NaN                         NaN   
1               0.0                   0.0                         1.0   

   mrentabilidad_slope_6  mrentabilidad_annual_slope_6  mcomisiones_slope_6  \
0                    NaN                           NaN                  NaN   
1                 984.71                      40616.29                -2.54   

   ...  Visa_madelantodolares_slope_6  Visa_fultimo_cierre_slope_6  \
0  ...                            NaN                          NaN   
1  ...                            0.0                          0.0   

   Visa_mpagado_slope_6  Visa_mpagospesos_slope_6  Visa_mpagosdolares_slope_6  \
0                   NaN                       NaN                         NaN   
1                   0.0                     210.2                         0.0   

   Visa_fechaalta_slope_6  Visa_mconsumototal_slope_6  Visa_cconsumos_slope_6  \
0                     NaN                         NaN                     NaN   
1                    28.0                      -17.48                     0.0   

   Visa_cadelantosefectivo_slope_6  Visa_mpagominimo_slope_6  
0                              NaN                       NaN  
1                              0.0                       0.0  

[2 rows x 152 columns]

In [6]:
%%sql
create or replace table competencia_02_fe as
SELECT * FROM competencia_02_lag_delta lag

LEFT JOIN competencia_02_avg avg 
ON lag.numero_de_cliente = avg.numero_de_cliente 
    AND lag.foto_mes = avg.foto_mes   
    
LEFT JOIN competencia_02_reg reg 
ON lag.numero_de_cliente = reg.numero_de_cliente 
    AND lag.foto_mes = reg.foto_mes     

Empty DataFrame
Columns: [Success]
Index: []

In [7]:
%%sql
SELECT * FROM competencia_02_fe
LIMIT 2

numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
0           81886526    202104               1            0         0   
1           81886526    202105               1            0         0   

   cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
0            39                 133        1023.44               6417.43   
1            39                 134        1359.11               7496.51   

   mcomisiones  ...  Visa_madelantodolares_slope_6  \
0       313.06  ...                            0.0   
1       200.13  ...                            0.0   

   Visa_fultimo_cierre_slope_6  Visa_mpagado_slope_6  \
0                    -0.535714                   0.0   
1                    -0.035714                   0.0   

   Visa_mpagospesos_slope_6  Visa_mpagosdolares_slope_6  \
0                   926.693                         0.0   
1                   862.040                         0.0   

   Visa_fechaalta_slope_6  Visa_mconsumototal_slope_6  Visa_cconsumos_slope_6  \
0               30.142857                   30.956000               -0.400000   
1               30.178571                   23.396977               -0.302326   

   Visa_cadelantosefectivo_slope_6  Visa_mpagominimo_slope_6  
0                              0.0               -138.246429  
1                              0.0               -276.492857  

[2 rows x 1056 columns]

Guardo un primer dataset con los meses de 202012 al 202107, (6 meses training, 1 test)

In [8]:
%%sql
create or replace table competencia_02_fe_202012 as
SELECT * FROM competencia_02_fe
WHERE foto_mes IN (202012,202101,202102,202103,202104,202105,202106,202107)

Empty DataFrame
Columns: [Success]
Index: []

In [11]:
%%sql
create or replace table clase_ternaria as
SELECT numero_de_cliente, foto_mes, clase_ternaria FROM competencia_02

Empty DataFrame
Columns: [Success]
Index: []

In [12]:
%%sql
create or replace table competencia_02_fe_202012 as
SELECT * FROM competencia_02_fe_202012 f
LEFT JOIN clase_ternaria p 
ON f.numero_de_cliente = p.numero_de_cliente 
    AND f.foto_mes = p.foto_mes  

Empty DataFrame
Columns: [Success]
Index: []

In [27]:
%%sql
create or replace table competencia_02_fe_202012 as
SELECT * EXCLUDE ("numero_de_cliente:1", "numero_de_cliente:2",	"numero_de_cliente:3","foto_mes:1","foto_mes:2" ,"foto_mes:3") FROM competencia_02_fe_202012

Empty DataFrame
Columns: [Success]
Index: []

In [23]:
# %%sql
# SELECT column_name
# FROM information_schema.columns
# WHERE table_name = 'competencia_02_fe_202012' AND column_name LIKE '%numero_de_cliente%';

column_name
0    numero_de_cliente
1  numero_de_cliente:1
2  numero_de_cliente:2
3  numero_de_cliente:3

Guardamos el dataset final:

In [28]:
%%sql
copy competencia_02_fe_202012 to 'C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/datasets/competencia_02_fe_202012.csv.gz' (FORMAT CSV, HEADER)

Empty DataFrame
Columns: [Success]
Index: []